In [12]:
!pip install emoji

In [13]:
import re
import joblib
import pandas as pd
from scipy.sparse import hstack
import pickle
import sys
import emoji

In [14]:
# Carregar listas de palavras tóxicas e não tóxicas usando pickle
with open('toxic_words.pkl', 'rb') as file:
    toxic_words_set = pickle.load(file)

with open('non_toxic_words.pkl', 'rb') as file:
    non_toxic_words_set = pickle.load(file)


In [15]:
# Carregar vetorizador e modelo

tfidf_vectorizer_1 = joblib.load('tfidf_vectorizer.pkl')
lg_model_1 = joblib.load('lg_model.pkl')

In [16]:
def remove_emoji(text):
    if text:
        return emoji.get_emoji_regexp().sub('', text)
    return text

# Função para remover caracteres que não são do alfabeto português
def remover_chars_n_pt(text):
    return re.sub(r'[^\w\s]', '', text)

# Funções de pré-processamento
def clean_text(text):
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'http\S+', '', text)  # Remove links
    text = remover_chars_n_pt(text)  # Remove non-Portuguese characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

def preprocess_text(text):
    text = clean_text(text)
    return text

def palavras_toxicas(text):
    for word in text.split():
        if word in toxic_words_set:
            return 1
    return 0

def numero_palavras_toxicas(text):
    count = 0
    for word in text.split():
        if word in toxic_words_set:
            count += 1
    return count

def palavras_n_toxicas(text):
    for word in text.split():
        if word in non_toxic_words_set:
            return 1
    return 0

def numero_palavras_n_toxicas(text):
    count = 0
    for word in text.split():
        if word in non_toxic_words_set:
            count += 1
    return count

def palavras_neutras(text):
    words = text.split()
    for word in words:
        if word in toxic_words_set and word in non_toxic_words_set:
            return 1
    return 0

def numero_palavras_neutras(text):
    words = text.split()
    count = 0
    for word in words:
        if word in toxic_words_set and word in non_toxic_words_set:
            count += 1
    return count



In [21]:
def predict_toxicity(tweet):
    # Pré-processar o texto
    tweet = preprocess_text(tweet)

    # Criar features
    features = {
        'text': tweet,
        'palavras tóxicas': palavras_toxicas(tweet),
        'número de palavras tóxicas': numero_palavras_toxicas(tweet),
        'palavras não tóxicas': palavras_n_toxicas(tweet),
        'número de palavras não tóxicas': numero_palavras_n_toxicas(tweet),
        'palavras neutras': palavras_neutras(tweet),
        'número de palavras neutras': numero_palavras_neutras(tweet),
        'número de palavras': len(tweet),
        'número de letras': len(tweet.split())
    }
     # Transformar texto usando TF-IDF
    X_t = tfidf_vectorizer_1.transform([tweet])

    # Criar DataFrame com outras features
    X_others = pd.DataFrame([features]).drop(columns=['text'])

    # Combinar as features
    X = hstack([X_t, X_others])

    # Fazer a predição
    prediction = lg_model_1.predict(X)

    return prediction[0]


In [26]:
#Função principal para entrada do usuário
if __name__ == "__main__":
    tweet = input("Digite o tweet: ")
    prediction = predict_toxicity(tweet)
    if prediction == 1:
        print("\033[1;31mO tweet é tóxico\033[0m")
    else:
        print("\033[1;32mO tweet é não tóxico\033[0m")



Digite o tweet: vai se fudeeee te amo demais meu amigo
O tweet é não tóxico
